In [42]:
from pyspark.sql import pandas
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
conf.set("spark.sql.execution.arrow.enabled","true")

spark = SparkSession.builder \
.appName("CURSOS") \
.config(conf=conf) \
.getOrCreate()

sqlCtx = SQLContext(spark)

22/06/17 15:47:39 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/06/17 15:47:39 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/06/17 15:47:39 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
/usr/local/lib/python3.7/site-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning
22/06/17 15:47:39 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.en

In [43]:
from pyspark.sql import functions as F
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql.functions import sum, max, min, avg, count, mean, round, col
import json

In [55]:
%run -i funcoes.ipynb

s3 = Aws_importer()
f_pandas = Pandas_Spark()

In [45]:
#schema for structure
customType = StructType([
    StructField('grau',StringType()),
    StructField('turno',StringType()),
    StructField('mensalidade',FloatType()),
    StructField('bolsa_integral_cotas',IntegerType()),
    StructField('bolsa_integral_ampla',IntegerType()),
    StructField('bolsa_parcial_cotas',IntegerType()),
    StructField('bolsa_parcial_ampla',IntegerType()),
    StructField('curso_id',IntegerType()),
    StructField('curso_busca',StringType()),
    StructField('cidade_busca',StringType()),
    StructField('uf_busca',StringType()),
    StructField('cidade_filtro',StringType()),
    StructField('universidade_nome',StringType()),
    StructField('campus_nome',StringType()),
    StructField('campus_external_id',IntegerType()),
    StructField('nome',StringType()),
    StructField('nota_integral_ampla',FloatType()),
    StructField('nota_integral_cotas',FloatType()),
    StructField('nota_parcial_ampla',FloatType()),
    StructField('nota_parcial_cotas',FloatType()),
])

In [46]:
sparkDF = spark.read.option('delimiter', ';').option('header', 'true').csv('s3a://alex-group2-sptech-bucket-brutos/cursos-prouni.csv',schema=customType)

22/06/17 15:47:53 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


In [47]:
sparkDF.show()

+------------+--------+-----------+--------------------+--------------------+-------------------+-------------------+--------+--------------------+-------------------+--------+----------------+--------------------+--------------------+------------------+--------------------+-------------------+-------------------+------------------+------------------+
|        grau|   turno|mensalidade|bolsa_integral_cotas|bolsa_integral_ampla|bolsa_parcial_cotas|bolsa_parcial_ampla|curso_id|         curso_busca|       cidade_busca|uf_busca|   cidade_filtro|   universidade_nome|         campus_nome|campus_external_id|                nome|nota_integral_ampla|nota_integral_cotas|nota_parcial_ampla|nota_parcial_cotas|
+------------+--------+-----------+--------------------+--------------------+-------------------+-------------------+--------+--------------------+-------------------+--------+----------------+--------------------+--------------------+------------------+--------------------+-----------------

In [48]:
#droping useless column
sparkDF = sparkDF.drop('curso_id','cidade_filtro','campus_external_id','campus_nome')

In [49]:
sparkDF.printSchema()

root
 |-- grau: string (nullable = true)
 |-- turno: string (nullable = true)
 |-- mensalidade: float (nullable = true)
 |-- bolsa_integral_cotas: integer (nullable = true)
 |-- bolsa_integral_ampla: integer (nullable = true)
 |-- bolsa_parcial_cotas: integer (nullable = true)
 |-- bolsa_parcial_ampla: integer (nullable = true)
 |-- curso_busca: string (nullable = true)
 |-- cidade_busca: string (nullable = true)
 |-- uf_busca: string (nullable = true)
 |-- universidade_nome: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- nota_integral_ampla: float (nullable = true)
 |-- nota_integral_cotas: float (nullable = true)
 |-- nota_parcial_ampla: float (nullable = true)
 |-- nota_parcial_cotas: float (nullable = true)



In [50]:
#null values
sparkDF = sparkDF.fillna({'mensalidade':0})
sparkDF = sparkDF.fillna({'bolsa_integral_cotas':0})
sparkDF = sparkDF.fillna({'bolsa_integral_ampla':0})
sparkDF = sparkDF.fillna({'bolsa_parcial_cotas':0})
sparkDF = sparkDF.fillna({'bolsa_parcial_ampla':0})
sparkDF = sparkDF.fillna({'nota_integral_ampla':0})
sparkDF = sparkDF.fillna({'nota_integral_cotas':0})
sparkDF = sparkDF.fillna({'nota_parcial_ampla':0})
sparkDF = sparkDF.fillna({'nota_parcial_cotas':0})

In [51]:
pdf = sparkDF.toPandas()

/usr/local/lib/python3.7/site-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 1.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)
/usr/local/lib/python3.7/site-packages/pyspark/sql/types.py:1422: RuntimeWarning: coroutine 'InteractiveShell.run_cell_async' was never awaited
  row = Row(*values)


In [52]:
pdf

,grau,turno,mensalidade,bolsa_integral_cotas,bolsa_integral_ampla,bolsa_parcial_cotas,bolsa_parcial_ampla,curso_busca,cidade_busca,uf_busca,universidade_nome,nome,nota_integral_ampla,nota_integral_cotas,nota_parcial_ampla,nota_parcial_cotas
0,Bacharelado,Integral,9999.990234,15,14,0,0,Medicina,Campo Grande,MS,Universidade Anhanguera - UNIDERP,Medicina,740.219971,726.460022,0.000000,0.000000
1,Bacharelado,Noturno,9836.400391,1,0,0,0,Enfermagem,Crateus,CE,Faculdade Princesa do Oeste - FPO,Enfermagem,663.359985,0.000000,0.000000,0.000000
2,Bacharelado,Integral,9715.610352,2,5,6,10,Medicina,Sao Paulo,SP,Universidade Cidade de São Paulo - UNICID,Medicina,739.619995,738.080017,738.960022,718.640015
3,Bacharelado,Noturno,9689.339844,3,2,0,0,Psicologia,Crateus,CE,Faculdade Princesa do Oeste - FPO,Psicologia,651.000000,652.219971,0.000000,0.000000
4,Bacharelado,Integral,9674.339844,4,1,5,2,Medicina,Rio Branco,AC,Faculdade Barão do Rio Branco - FAB,Medicina,758.320007,723.940002,734.919983,711.260010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41442,Licenciatura,Curso a Distância,149.000000,1,0,0,0,Sociologia,Paraiso do Tocantins,TO,Universidade Paulista - UNIP,Sociologia,502.359985,0.000000,0.000000,0.000000
41443,Tecnológico,Curso a Distância,144.000000,1,2,2,5,Marketing,Sao VIcente,SP,Faculdade de São VIcente - FSV,Marketing,533.340027,450.000000,450.000000,450.000000
41444,Tecnológico,Vespertino,139.000000,1,0,0,0,Gestão de Recursos Humanos,Brasilia,DF,Centro Universitário Planalto do Distrito Fede...,Gestão de Recursos Humanos,580.760010,0.000000,0.000000,0.000000
41445,Bacharelado,Curso a Distância,139.000000,1,0,0,0,Serviço Social,Vitoria,ES,Universidade Paulista - UNIP,Serviço Social,548.260010,0.000000,0.000000,0.000000


In [53]:
#chaging the cities names
abvEST = ["AC","AL","AP","AM","BA","CE","ES","GO","MA","MT","MS","MG","PA","PB","PR","PE","PI","RJ","RN","RS","RO","RR","SC","SP","SE","TO","DF"]
estados = ["Acre","Alagoas","Amapa","Amazonas","Bahia","Ceara","Espirito Santo","Goias","Maranhao","Mato Grosso","Mato Grosso do Sul","Minas Gerais","Para","Paraiba","Parana","Pernambuco","Piaui","Rio de Janeiro","Rio Grande do Norte","Rio Grande do Sul","Rondonia","Roraima","Santa Catarina","Sao Paulo","Sergipe","Tocantins","Brasilia"]
for index, i in enumerate(estados):
    pdf = pdf.replace(abvEST[index],i)

In [54]:
pdf

,grau,turno,mensalidade,bolsa_integral_cotas,bolsa_integral_ampla,bolsa_parcial_cotas,bolsa_parcial_ampla,curso_busca,cidade_busca,uf_busca,universidade_nome,nome,nota_integral_ampla,nota_integral_cotas,nota_parcial_ampla,nota_parcial_cotas
0,Bacharelado,Integral,9999.990234,15,14,0,0,Medicina,Campo Grande,Mato Grosso do Sul,Universidade Anhanguera - UNIDERP,Medicina,740.219971,726.460022,0.000000,0.000000
1,Bacharelado,Noturno,9836.400391,1,0,0,0,Enfermagem,Crateus,Ceara,Faculdade Princesa do Oeste - FPO,Enfermagem,663.359985,0.000000,0.000000,0.000000
2,Bacharelado,Integral,9715.610352,2,5,6,10,Medicina,Sao Paulo,Sao Paulo,Universidade Cidade de São Paulo - UNICID,Medicina,739.619995,738.080017,738.960022,718.640015
3,Bacharelado,Noturno,9689.339844,3,2,0,0,Psicologia,Crateus,Ceara,Faculdade Princesa do Oeste - FPO,Psicologia,651.000000,652.219971,0.000000,0.000000
4,Bacharelado,Integral,9674.339844,4,1,5,2,Medicina,Rio Branco,Acre,Faculdade Barão do Rio Branco - FAB,Medicina,758.320007,723.940002,734.919983,711.260010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41442,Licenciatura,Curso a Distância,149.000000,1,0,0,0,Sociologia,Paraiso do Tocantins,Tocantins,Universidade Paulista - UNIP,Sociologia,502.359985,0.000000,0.000000,0.000000
41443,Tecnológico,Curso a Distância,144.000000,1,2,2,5,Marketing,Sao VIcente,Sao Paulo,Faculdade de São VIcente - FSV,Marketing,533.340027,450.000000,450.000000,450.000000
41444,Tecnológico,Vespertino,139.000000,1,0,0,0,Gestão de Recursos Humanos,Brasilia,Brasilia,Centro Universitário Planalto do Distrito Fede...,Gestão de Recursos Humanos,580.760010,0.000000,0.000000,0.000000
41445,Bacharelado,Curso a Distância,139.000000,1,0,0,0,Serviço Social,Vitoria,Espirito Santo,Universidade Paulista - UNIP,Serviço Social,548.260010,0.000000,0.000000,0.000000


In [58]:
# sparkDF = spark.createDataFrame(pdf, schema=customType2)
sparkDF= f_pandas.pandas_to_spark(pandas_df=pdf)

In [57]:
sparkDF.printSchema()

root
 |-- grau: string (nullable = true)
 |-- turno: string (nullable = true)
 |-- mensalidade: float (nullable = true)
 |-- bolsa_integral_cotas: integer (nullable = true)
 |-- bolsa_integral_ampla: integer (nullable = true)
 |-- bolsa_parcial_cotas: integer (nullable = true)
 |-- bolsa_parcial_ampla: integer (nullable = true)
 |-- curso_busca: string (nullable = true)
 |-- cidade_busca: string (nullable = true)
 |-- uf_busca: string (nullable = true)
 |-- universidade_nome: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- nota_integral_ampla: float (nullable = true)
 |-- nota_integral_cotas: float (nullable = true)
 |-- nota_parcial_ampla: float (nullable = true)
 |-- nota_parcial_cotas: float (nullable = true)



In [18]:
#data pre-processed
s3.save_s3_data(sparkDF,'preprocessado','CURSOS_PROUNI')
s3.save_s3_data(sparkDF,'cliente','CURSOS_PROUNI')
s3.save_data_client_side(sparkDF,'CURSOS')

22/06/17 15:27:54 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/17 15:27:55 WARN TaskSetManager: Stage 4 contains a task of very large size (3670 KiB). The maximum recommended task size is 1000 KiB.
22/06/17 15:27:55 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/17 15:27:55 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/17 15:27:59 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/17 15:27:59 WARN TaskSetManager: Stage 5 contains a task of very large size (3670 KiB). The maximum recommended task size is 1000 KiB.
22/06/17 15:27:59 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/17 15:27:59 

In [19]:
def soma_col(coluna_a,coluna_b):
    col_sum = coluna_a+coluna_b
    return col_sum

funcao_soma = F.udf(soma_col, IntegerType())

In [20]:
#sugestion table for cliente
sparkDF = sparkDF.withColumn("bolsa_integral_total", funcao_soma("bolsa_integral_cotas","bolsa_integral_ampla"))
sparkDF = sparkDF.withColumn("bolsa_parcial_total", funcao_soma("bolsa_parcial_cotas","bolsa_parcial_ampla"))

22/06/17 15:28:18 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


In [21]:
sugestionDF = sparkDF.select('grau','turno','uf_busca','universidade_nome','nome','bolsa_integral_total','bolsa_parcial_total')

In [22]:
sugestionDF.show()

22/06/17 15:28:39 WARN TaskSetManager: Stage 7 contains a task of very large size (3670 KiB). The maximum recommended task size is 1000 KiB.


+------------+--------+------------------+--------------------+--------------------+--------------------+-------------------+
|        grau|   turno|          uf_busca|   universidade_nome|                nome|bolsa_integral_total|bolsa_parcial_total|
+------------+--------+------------------+--------------------+--------------------+--------------------+-------------------+
| Bacharelado|Integral|Mato Grosso do Sul|Universidade Anha...|            Medicina|                  29|                  0|
| Bacharelado| Noturno|             Ceara|Faculdade Princes...|          Enfermagem|                   1|                  0|
| Bacharelado|Integral|         Sao Paulo|Universidade Cida...|            Medicina|                   7|                 16|
| Bacharelado| Noturno|             Ceara|Faculdade Princes...|          Psicologia|                   5|                  0|
| Bacharelado|Integral|              Acre|Faculdade Barão d...|            Medicina|                   5|             

In [23]:
#data pre-processed sugestion
s3.save_s3_data(sugestionDF,'preprocessado','CURSOS_PROUNI_TOTAL_BOLSAS')
s3.save_s3_data(sugestionDF,'cliente','CURSOS_PROUNI_TOTAL_BOLSAS')
s3.save_data_client_side(sugestionDF,'CURSOS_TOTAL_BOLSAS')

22/06/17 15:30:21 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/17 15:30:22 WARN TaskSetManager: Stage 8 contains a task of very large size (3670 KiB). The maximum recommended task size is 1000 KiB.
22/06/17 15:30:22 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/17 15:30:22 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/17 15:30:26 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/17 15:30:27 WARN TaskSetManager: Stage 9 contains a task of very large size (3670 KiB). The maximum recommended task size is 1000 KiB.
22/06/17 15:30:27 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/17 15:30:27 

In [24]:
spark.stop()